## Preamble

Run these if notebook is in a new environment, otherwise just skip to `Import modules`

#### Boot cell

Skip if `module_update.py` is in same folder as notebook, next cell downloads this file which in turn will download other files.

In [1]:
!curl -s "https://raw.githubusercontent.com/Yoonsen/Modules/master/module_update.py" > "module_update.py"

#### Get essentials

Import text and graph modules for analysis, and css for display

In [2]:
from module_update import update, css

In [3]:
css()

# Import modules

Aktiver cellen nedenfor for å få alt til å virke.

In [4]:
update('nbtext', overwrite=True)
import nbtext as nb
from IPython.display import HTML
import requests
from bs4 import BeautifulSoup


def mods_digibok(urn, kind='marcxml'):
    """find information about serial number urn (or full urn) for books using NB API. 
    Parameter 'kind' is one of mods, marcxml or dublincore """
    
    if not str(urn).startswith('URN'):
        urnstr = "URN:NBN:no-nb_digibok_{urn}".format(urn=urn)
    r = requests.get("https://api.nb.no:443/catalog/v1/metadata/{ident}/{kind}".format(ident=urnstr, kind=kind))
    return r.text

def translator(urn):
    """Finds the translator for a book"""
    item = BeautifulSoup(mods_digibok(urn, kind='mods'), 'lxml')
    trans = []
    navn = item.find_all('name', {'type':'personal'})
    for i in navn:

        try:
            if i.roleterm.text == 'trl':
                trans.append(i.namepart.text)
        except:
            'nn'
    return ';'.join(trans)
    

Updated file `C:\Users\larsj\Documents\GitHub\Oversettelse\nbtext.py`

## Build corpus

A corpus definition is a set of URNs. Those can be obtained in various ways and stored as text or in MS-Word file.


### Define using metadata 

Using author name, titles, year, dewey, subject and gender. Check out [Webdewey](http://deweysearchno.pansoft.de/webdeweysearch/index.html) for more descriptions.

In [5]:
korpus = nb.get_urn({
                     'year':1950, 
                     'next':5,
                     'trans':'eng',
                     'limit':3000})
len(korpus)

1199

# Hent oversetterne

Her defineres korpuset som en liste over bøker, med informasjon om oversettere. Er korpuset stort (flere hundre) vil det ta noen minutter å hente alle oversetterne, så det kan være lurt å kutte ned størrelsen litt. Inntil vi får på plass en rask måte å gjøre det på, får det her duge. Det er bare å smøre seg med tålmodighet.

In [11]:
rammen = nb.frame(nb.metadata(korpus), ['urn', 'forfatter','år','tittel','språk','forlag', 'original' ]).sort_values(by='tittel')
rammen = rammen.set_index('urn')
rammen.index = rammen.index.astype(str)



## Se på hele korpuset under ett som HTML

Oversetterne ligger i en egen kolonne `oversetter`. Fyll in feltet bak `by` med et kolonnenavn for å sortere på verdiene i den kolonnen.

In [12]:
HTML(rammen.sort_values(by = 'forfatter').to_html())

,forfatter,år,tittel,språk,forlag,original
urn,,,,,,
2011031720018,,1950,"""Gud er sanndru""",Watchtower Bible and Tract Society,nob,eng
2014030426005,,1954,Malermester Langøre,Tiden,nob,eng
2014020328033,,1954,Dyrene i Arken,Tiden,nob,eng
2014020626013,,1953,Mikke Mus på biltur,Tiden,nob,eng
2014020626017,,1955,Hvor er Petter Etterstad?,Tiden,nob,eng
2016102648548,,1955,Hva har religionen gjort for menneskeheten?,,nob,eng
2011110708036,,1951,De tre bjørnene,Tiden,nob,eng
2008042800003,,1950,Verdenserklæringen om menneskerettigheter,Norsk samband for de Forente nasjoner,nob,eng
2014103007533,,1955,Verden vi lever i,Reinert,nob,eng


## Search corpus for concordances

In [13]:
from random import sample

In [20]:
nb.urn_concordance(word= ['gråtende', 'smilende'], urns=sample(korpus,10))

"På Forsyte-børsen, Galsworthy, John, 1954",var snart likeså tynn og,gråtende,og iull av klvpernerker Born
"På Forsyte-børsen, Galsworthy, John, 1954","opp , fulgt av den",smilende,Smith . « Vent her
"Den syvende patronhylsen, Coxe, George Harmon, 1952",etter seg . McCann så,smilende,på Lafferty .
"Indianeren Hjortefot, Ellis, Edward S., 1953",", derfor snudde han seg",smilende,mot Ned .
"Lærstrømpe, Cooper, James Fenimore, 1954",", og han vendte seg",smilende,mot datteren . — Se
"Lærstrømpe, Cooper, James Fenimore, 1954",vilje . Marmaduke grep ham,smilende,i hånden . Den sårede
"I biskopens vogn, Michelson, Miriam, 1953",jeg døren opp og sto,smilende,foran ham . Joda .
"Makten og æren, Greene, Graham, 1950",", » og han våknet",gråtende,. Han hadde nok bare
"Makten og æren, Greene, Graham, 1950",Han smålo usikkert bak den,smilende,masken . « Det er


In [22]:
nb.get_urnkonk(['gråtende', 'smilende'], {'urns':sample(korpus, 50)})

"Til himlen vil jeg fare, Wilder, Thornton, 1951",var halvt leende og halvt,gråtende,. Hun spurte meg om
"Til himlen vil jeg fare, Wilder, Thornton, 1951","rystet på hodet , fremdeles",smilende,.
"Til himlen vil jeg fare, Wilder, Thornton, 1951",Brush sa,smilende,til mrs . Efrim :
"Til himlen vil jeg fare, Wilder, Thornton, 1951",Miss Colloquer kom,smilende,inn . « Dr. Bowie
"Flammende sverd, Caldwell, Taylor, 1950",Kurelen var forvandlet til en,gråtende,", ynkelig stakkar . De"
"Flammende sverd, Caldwell, Taylor, 1950",inn i leiren igjen med,gråtende,kvinner bak seg på hesteryggen
"Flammende sverd, Caldwell, Taylor, 1950",av teltvogner skramlet fram med,gråtende,kvinner og barn . Men
"Flammende sverd, Caldwell, Taylor, 1950",og ba og trykte sine,gråtende,barn til brystet . Kvegflokkene
"Flammende sverd, Caldwell, Taylor, 1950",banet seg vei gjennom den,gråtende,og jamrende menneskemassen og fant
"Flammende sverd, Caldwell, Taylor, 1950","med dyrehuder , og tørret",smilende,sin brede forvridde munn på
"Flammende sverd, Caldwell, Taylor, 1950",Langsomt og,smilende,hevet Kurelen dolken og satte


In [24]:
nb.word_paradigm('smilende')

[['adj', ['smilende', 'smilt', 'smilte']],
 ['verb', ['smil', 'smile', 'smiler', 'smiles', 'smilt', 'smilte']]]

In [26]:
nb.urn_concordance(word= ['smil', 'smile', 'smiler', 'smiles', 'smilt', 'smilte'], urns=sample(korpus,10))

"Havefesten, Mansfield, Katherine, 1950",", for det har et",smil,ifrå en drøm i seg
"Havefesten, Mansfield, Katherine, 1950",tomme huset med et drømmende,smil,.
"Havefesten, Mansfield, Katherine, 1950","svulmet , med et unnselig",smil,.
"Havefesten, Mansfield, Katherine, 1950",", deres slu , dulgte",smil,; de var medlemmer av
"Havefesten, Mansfield, Katherine, 1950",et rutet handkle ; et,smil,svevet om leppene ; hun
"Havefesten, Mansfield, Katherine, 1950",? » Han prøvde å,smile,. « La oss glemme
"Havefesten, Mansfield, Katherine, 1950","melk og vann , måtte",smile,. « Dere bå ikke
"Havefesten, Mansfield, Katherine, 1950",hviske når han snakket og,smile,unnselig ; og hvor gildt
"Havefesten, Mansfield, Katherine, 1950",kunne ikke la være å,smile,barskt mens han møy *
"Havefesten, Mansfield, Katherine, 1950","legge kinn mot kinn ,",smile,inn i hverandres øyne .
"Havefesten, Mansfield, Katherine, 1950","moren som , mens speilet",smiler,", fester hennes snorer ,"
